# Neurocombat TOP and StrokeMRI harmonized datasets

Note this must be run in the `neuron` environment

## import libraries

In [ ]:
import os

import pandas as pd
import numpy as np

from neuroCombat import neuroCombat

## Let's see how we would apply this to our data
We will have to flip it on it's side to make it work

In [ ]:
filepath = '../open_work/internal_results/cleaned_pvc2s/' 
filename_mri = os.path.join(filepath,'StrokeMRI_pvc2c.csv') 
filename_top = os.path.join(filepath,'TOP_pvc2c.csv') 

In [ ]:
our_mri_data = pd.read_csv(filename_mri)
our_top_data = pd.read_csv(filename_top)

In [ ]:
print("Our StrokeMRI data is", len(our_mri_data), "patients")
print("Our TOP data is", len(our_top_data), "patients")

In [ ]:
our_top_data.head(3)

In [ ]:
our_mri_data.head(3)

In [ ]:
top_selection = our_top_data.drop(['Unnamed: 0'],axis=1)
mri_selection = our_mri_data.drop(['Unnamed: 0'],axis=1)

In [ ]:
#mri_selection.isna().sum()
#top_selection.isna().sum()

In [ ]:
mri_selection = mri_selection.set_index('participant_id')
top_selection = top_selection.set_index('participant_id')

In [ ]:
top_selection = top_selection.T
mri_selection = mri_selection.T
#mri_selection

In [ ]:
both_together = pd.concat([ mri_selection, top_selection], axis=1, join="inner")
both_together

In [ ]:
# make sure there are no NaNs
both_together.isna().sum().sum()

In [ ]:
# # save off csv
# both_together.to_csv('both_top_mri_together.csv')

In [ ]:
# make and save of csv of features only
features_only = both_together[2:]
#features_only.to_csv('features_only_top_mri.csv')

In [ ]:
features_only.T.columns

In [ ]:
dictionary_features_len = len(features_only.T.columns)
number = 0
made_keys = [] 
made_vals = [] 
for n in features_only.T.columns:
    
    made_keys.append(number)
    made_vals.append(n)
    number +=1

In [ ]:
feature_dict = dict(map(lambda i,j : (i,j) , made_keys,made_vals))

In [ ]:
ft = features_only.reset_index()
ft = ft.rename(columns={"index": "A"})
ft = ft.drop(['A'], axis=1)
ft = ft.dropna()

In [ ]:
bt= both_together.reset_index()
bt = bt.rename(columns={"index": "A"})
bt = bt.drop(['A'], axis=1)
bt = bt.dropna()

In [ ]:
bt.head(3)

In [ ]:
#ft.to_csv('ft_top_mri.csv')

In [ ]:
data = np.genfromtxt('ft_top_mri.csv', delimiter=",", skip_header=1)
data

In [ ]:
data = data[:, 1:]
#data

In [ ]:
# Specifying the batch (scanner variable) as well as a sex covariate to preserve:
first_columns_as_one = [1] * len(mri_selection.columns)
last_columns_as_two = [2] * len(top_selection.columns)
covars = {'batch':first_columns_as_one + last_columns_as_two,
          'sex':both_together.loc['sex',:].values.tolist(),
           'age':both_together.loc['age',:].values.tolist(),} 
covars = pd.DataFrame(covars) 

In [ ]:
#covars

In [ ]:
covars.shape

In [ ]:
data.shape

In [ ]:
# specifify sex as categorical
categorical_cols = ['sex']
# To specify the name of the variable that encodes for the scanner/batch covariate:
batch_col_mine = 'batch'

#Harmonization step:
data_combat = neuroCombat(dat=data,
    covars=covars,
    batch_col=batch_col_mine,
    categorical_cols=categorical_cols)["data"]

In [ ]:
data_combat

In [ ]:
data_combat.shape

In [ ]:
neurocombat = pd.DataFrame(data_combat)
neurocombat

In [ ]:
topper = bt.head(2)

In [ ]:
topper = topper.rename_axis(None, axis="columns")

In [ ]:
topper

In [ ]:
topper = topper.reset_index(drop=False)
topper = topper.rename(columns={"index": "char"})

In [ ]:
topper['char'][0] = 'age'
topper['char'][1] = 'sex'

In [ ]:
topper

In [ ]:
bottom = neurocombat.reset_index(drop=False)
bottom = bottom.rename(columns={"index": "char"})
bottom.columns = topper.columns
#bottom = bottom.set_index('char', inplace=True)
#set_index('Name', inplace=True)
bottom

In [ ]:
back_together = pd.concat([topper, bottom])
back_together = back_together.T
back_together

In [ ]:
new_header = back_together.iloc[0] #grab the first row for the header
back_together.columns = new_header #set the header row as the df header
back_together = back_together[1:]
back_together

In [ ]:
back_together.head(514).tail(10)

In [ ]:
back_together.tail(527).head(10)

In [ ]:
neuro_harm_top =back_together.tail(527)
neuro_harm_mri =back_together.head(514)

In [ ]:
neuro_harm_top = neuro_harm_top.rename(feature_dict, axis='columns')
neuro_harm_mri = neuro_harm_mri.rename(feature_dict, axis='columns')

In [ ]:
# neuro_harm_mri.to_csv('neuro_harm_mri.csv')
# neuro_harm_top.to_csv('neuro_harm_top.csv')

In [ ]:
# Investigate neuroharmoney results

In [ ]:
neuro_harm_mri.tail(3) 

## We see neuroharmoney added negative numbers 
let's compare to the original

In [ ]:
mri_selection.T.tail(3)

## let's see how many negative numbers we have

In [ ]:
number_columns = ['gm_vol',
                  'wm_vol',
                  'csf_vol',
                  'gm_ivc_ratio', 
                  'gmwm_ivc_ratio',
                  'wmh_vol',
                  'wmh_count',
                  'deepwm_b_cov',
                  'aca_b_cov',
                  'mca_b_cov',
                  'pca_b_cov',
                  'totalgm_b_cov',
                  'deepwm_b',
                  'aca_b',
                  'mca_b',
                  'pca_b',
                  'totalgm_b',]

In [ ]:
(neuro_harm_mri[number_columns] < 0).sum()

In [ ]:
#our_top_data.head(3)
big_top = our_top_data.rename(columns={"Unnamed: 0": "level_0"})
big_top.head(3)

In [ ]:
new_harm_top = neuro_harm_top.reset_index()
new_harm_top = new_harm_top.reset_index()
new_harm_top = new_harm_top.rename(columns={"index": "participant_id"})
new_harm_top.head(3) 

In [ ]:
# difference in harmonized and original - raw
raw_differences = big_top[number_columns] - new_harm_top[number_columns]

In [ ]:
# difference in harmonized and original - raw
differences = (big_top[number_columns] - new_harm_top[number_columns])/big_top[number_columns]
differences

In [ ]:
differences_n = differences.apply(pd.to_numeric) #

In [ ]:
differences_n.describe()

## Now we can use the versions of TOP with log base 10 revision of white matter hyperintensity count, as well as white matter hyperintensity  volume

In [ ]:
log_filepath = '../open_work/internal_results/loggy/' 
log_filename_mri = os.path.join(log_filepath,'stroke_loged.csv') 
log_filename_top = os.path.join(log_filepath,'top_loged.csv') 

In [ ]:
log_mri_data = pd.read_csv(log_filename_mri)
log_top_data = pd.read_csv(log_filename_top)

In [ ]:
#log_top_data.head(3)

In [ ]:
#log_mri_data.head(3)

In [ ]:
log_top_selection = log_top_data.drop(['Unnamed: 0'],axis=1)
log_mri_selection = log_mri_data.drop(['Unnamed: 0'],axis=1)

In [ ]:
log_mri_selection = log_mri_selection.set_index('participant_id')
log_top_selection = log_top_selection.set_index('participant_id')

In [ ]:
log_top_selection = log_top_selection.T
log_mri_selection = log_mri_selection.T
#log_mri_selection

In [ ]:
log_both_together = pd.concat([log_mri_selection, log_top_selection], axis=1, join="inner")
log_both_together

In [ ]:
# make sure there are no NaNs
log_both_together.isna().sum().sum()

In [ ]:
# make and save of csv of features only
log_features_only = log_both_together[2:]
#log_features_only.to_csv('log_features_only_top_mri.csv')

In [ ]:
#log_features_only.T.columns

In [ ]:
#features_only.T.columns
## so we can use same features dictionary

In [ ]:
log_ft = log_features_only.reset_index()
log_ft = log_ft.rename(columns={"index": "A"})
log_ft = log_ft.drop(['A'], axis=1)
log_ft = log_ft.dropna()

In [ ]:
log_bt = log_both_together.reset_index()
log_bt = log_bt.rename(columns={"index": "A"})
log_bt = log_bt.drop(['A'], axis=1)
log_bt = log_bt.dropna()

In [ ]:
log_ft.to_csv('log_ft_top_mri.csv')

In [ ]:
log_data = np.genfromtxt('log_ft_top_mri.csv', delimiter=",", skip_header=1)
#log_data

In [ ]:
log_data = log_data[:, 1:]
#log_data

In [ ]:
# Specifying the batch (scanner variable) as well as a sex covariate to preserve:
first_columns_as_one = [1] * len(log_mri_selection.columns)
last_columns_as_two = [2] * len(log_top_selection.columns)
covars = {'batch':first_columns_as_one + last_columns_as_two,
          'sex':log_both_together.loc['sex',:].values.tolist(),
           'age':log_both_together.loc['age',:].values.tolist(),} 
covars = pd.DataFrame(covars) 

In [ ]:
# specifify sex as categorical
categorical_cols = ['sex']
# To specify the name of the variable that encodes for the scanner/batch covariate:
batch_col_mine = 'batch'

#Harmonization step:
log_data_combat = neuroCombat(dat=log_data,
    covars=covars,
    batch_col=batch_col_mine,
    categorical_cols=categorical_cols)["data"]

In [ ]:
log_data_combat

In [ ]:
log_neurocombat = pd.DataFrame(data_combat)
log_neurocombat

In [ ]:
log_topper = log_bt.head(2)

In [ ]:
log_topper = log_topper.rename_axis(None, axis="columns")
log_topper

In [ ]:
log_topper = log_topper.reset_index(drop=False)
log_topper = log_topper.rename(columns={"index": "char"})

In [ ]:
log_topper['char'][0] = 'age'
log_topper['char'][1] = 'sex'

In [ ]:
log_bottom = log_neurocombat.reset_index(drop=False)
log_bottom = log_bottom.rename(columns={"index": "char"})
log_bottom.columns = topper.columns
log_bottom

In [ ]:
log_back_together = pd.concat([log_topper, log_bottom])
log_back_together = log_back_together.T
log_back_together

In [ ]:
log_new_header = log_back_together.iloc[0] #grab the first row for the header
log_back_together.columns = new_header #set the header row as the df header
log_back_together = log_back_together[1:]
log_back_together

In [ ]:
log_back_together.head(514).tail(5)

In [ ]:
log_back_together.tail(527).head(5)

In [ ]:
log_neuro_harm_top =log_back_together.tail(527)
log_neuro_harm_mri =log_back_together.head(514)

In [ ]:
log_neuro_harm_top = log_neuro_harm_top.rename(feature_dict, axis='columns')
log_neuro_harm_mri = log_neuro_harm_mri.rename(feature_dict, axis='columns')

In [ ]:
# log_neuro_harm_mri.to_csv('log_neuro_harm_mri.csv')
# log_neuro_harm_top.to_csv('log_neuro_harm_top.csv')

In [ ]:
#log_neuro_harm_mri
(log_neuro_harm_mri[number_columns] < 0).sum()

In [ ]:
## There are still negative numbers in the outcome...but we can test to see if there is a difference

In [ ]:
log_neuro_harm_mri.head(3)# - neuro_harm_mri

In [ ]:
neuro_harm_mri.head(3)

In [ ]:
dif_log_to_reg = log_neuro_harm_mri[number_columns] - neuro_harm_mri[number_columns]
dif_log_to_reg 

In [ ]:
dif_log_to_reg.sum().sum()

# So changing the two columns to theirlog made literally no difference in the harmarmonized outcomes? Is there a bug??